In [99]:
import pandas as pd
import numpy as np
import string
import polars as pl

num_rows = 9000000
num_cols = 4
str_lenght = 5

def make_df(num_rows: int, num_dtype, cat_dtype):
    # Generate numerical data
    numerical_data = np.random.randint(num_rows, size=(num_rows, 5))
    numerical_headers = [f'num_col_{item}' for item in range(5)]

    
    # Generate categorical data, then add to dataframe
    categories = np.random.choice(['A', 'B', 'C'], size=(num_rows, 3))
    categorical_data = {f"cat_col_{i}": categories[:, i] for i in range(3)}
    # Make df then coerce dtypes
    df = pd.DataFrame().from_dict({**categorical_data})
    df[numerical_headers] = pd.DataFrame(numerical_data).astype(num_dtype)
    df[list(categorical_data.keys())] = df[list(categorical_data.keys())].astype(cat_dtype)
    
    return df

def add_string_data(df, num_rows, num_cols, string_length, dtype):
    # Define the length of each random string
    random_chars = np.random.choice(list(string.ascii_letters), size=(num_rows, num_cols * string_length))
    s_matrix = pd.DataFrame(random_chars.view('U' + str(string_length)).reshape(num_rows, num_cols), dtype=dtype)
    s_columns = [f'str_col_{num}' for num in range(num_cols)]

    df[s_columns] = s_matrix
    return df

df_arrow = make_df(num_rows, 'uint64[pyarrow]', 'category').pipe(add_string_data, num_rows, num_cols, str_lenght, 'string[pyarrow]')
df_np = make_df(num_rows, np.int64, 'category').pipe(add_string_data, num_rows, num_cols, str_lenght, 'object')


In [100]:
df_arrow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000000 entries, 0 to 8999999
Data columns (total 12 columns):
 #   Column     Dtype          
---  ------     -----          
 0   cat_col_0  category       
 1   cat_col_1  category       
 2   cat_col_2  category       
 3   num_col_0  uint64[pyarrow]
 4   num_col_1  uint64[pyarrow]
 5   num_col_2  uint64[pyarrow]
 6   num_col_3  uint64[pyarrow]
 7   num_col_4  uint64[pyarrow]
 8   str_col_0  string         
 9   str_col_1  string         
 10  str_col_2  string         
 11  str_col_3  string         
dtypes: category(3), string(4), uint64[pyarrow](5)
memory usage: 678.1 MB


In [101]:
df_np.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000000 entries, 0 to 8999999
Data columns (total 12 columns):
 #   Column     Dtype   
---  ------     -----   
 0   cat_col_0  category
 1   cat_col_1  category
 2   cat_col_2  category
 3   num_col_0  int64   
 4   num_col_1  int64   
 5   num_col_2  int64   
 6   num_col_3  int64   
 7   num_col_4  int64   
 8   str_col_0  object  
 9   str_col_1  object  
 10  str_col_2  object  
 11  str_col_3  object  
dtypes: category(3), int64(5), object(4)
memory usage: 643.7+ MB


In [102]:
# Make polar lazy- and dataframe
pf = pl.DataFrame(df_np)
lf = pl.LazyFrame(df_np)

In [103]:
# Show dtypes
pf.head()

cat_col_0,cat_col_1,cat_col_2,num_col_0,num_col_1,num_col_2,num_col_3,num_col_4,str_col_0,str_col_1,str_col_2,str_col_3
cat,cat,cat,i64,i64,i64,i64,i64,str,str,str,str
"""B""","""A""","""B""",6384963,2070407,1212543,6269157,80888,"""VzTJP""","""hUZHs""","""YhFhX""","""OuiBm"""
"""A""","""A""","""C""",8469874,1157160,1074855,3081276,7332521,"""vlmby""","""aezxX""","""ufGFk""","""pbsrU"""
"""B""","""A""","""B""",7461676,3495433,8953809,6203175,3692838,"""PTPrT""","""RPRpb""","""RpKyh""","""ljekI"""
"""B""","""A""","""C""",5294404,7864598,6824553,5676433,3753246,"""DjzSA""","""Snaju""","""vgmkJ""","""YmrrW"""
"""C""","""C""","""B""",7024990,6165127,6498646,7205152,4116379,"""tLWtW""","""XMjan""","""ALmiw""","""ZnEdq"""


In [104]:
lf.head().collect()

cat_col_0,cat_col_1,cat_col_2,num_col_0,num_col_1,num_col_2,num_col_3,num_col_4,str_col_0,str_col_1,str_col_2,str_col_3
cat,cat,cat,i64,i64,i64,i64,i64,str,str,str,str
"""B""","""A""","""B""",6384963,2070407,1212543,6269157,80888,"""VzTJP""","""hUZHs""","""YhFhX""","""OuiBm"""
"""A""","""A""","""C""",8469874,1157160,1074855,3081276,7332521,"""vlmby""","""aezxX""","""ufGFk""","""pbsrU"""
"""B""","""A""","""B""",7461676,3495433,8953809,6203175,3692838,"""PTPrT""","""RPRpb""","""RpKyh""","""ljekI"""
"""B""","""A""","""C""",5294404,7864598,6824553,5676433,3753246,"""DjzSA""","""Snaju""","""vgmkJ""","""YmrrW"""
"""C""","""C""","""B""",7024990,6165127,6498646,7205152,4116379,"""tLWtW""","""XMjan""","""ALmiw""","""ZnEdq"""


### Sortering av kolonner

In [105]:
cols=['num_col_0','str_col_1'] # columns to be used for sorting

In [106]:
%%timeit
df_np.sort_values(by=cols,ascending=True)

46.8 s ± 720 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
%%timeit
df_arrow.sort_values(by=cols,ascending=True)

30.1 s ± 7.24 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [108]:
%%timeit
pf.sort(cols,descending=False)

2.13 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [109]:
%%timeit
lf.sort(by=cols, descending=False)

11.5 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [110]:
%%timeit
lf.sort(by=cols, descending=False).collect()

2.14 s ± 84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [111]:
%%timeit
lf.sort(by=cols, descending=False).sort(by=['cat_col_1', 'str_col_3'], descending=False).collect()

23 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Filtrering

In [112]:
# Pick out datasets
num_cols = df_np.columns[df_np.columns.str.contains('num')].to_list()
cat_cols = df_np.columns[df_np.columns.str.contains('cat')].to_list()
str_cols = df_np.columns[df_np.columns.str.contains('str')].to_list()

Hvor mange unike strings finnes i dataene i rader der num_col_1 er mindre enn 10000?

In [113]:
%%timeit
df_np.loc[df_np['num_col_1']<=10000, str_cols].nunique()

1.71 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [114]:
%%timeit
df_arrow.loc[df_arrow['num_col_1']<=10000, str_cols].nunique()

67.3 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [115]:
%%timeit
pf.filter(pl.col("num_col_1") <= 10000).select(pl.col(str_cols).n_unique()) 

13.8 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [116]:
%%timeit
lf.filter(pl.col("num_col_1") <= 10000).select(pl.col(str_cols).n_unique()).collect()

12.3 ms ± 278 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Hva er gjennomsnittsverdiene for tallkolonnene for kategori A og kategori B?

In [117]:
%%timeit
mask = ((df_np['cat_col_1'] == 'A') & (df_np['cat_col_2'] == 'B'))
df_np.loc[mask, num_cols].mean()

233 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [118]:
%%timeit
mask = ((df_arrow['cat_col_1'] == 'A') & (df_arrow['cat_col_2'] == 'B'))
df_arrow.loc[mask, num_cols].mean()

63 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [119]:
%%timeit
(pf.filter(pl.col("cat_col_1") == 'A',
          pl.col('cat_col_2') == 'B')
 .select(pl.col(num_cols).mean()))

99.3 ms ± 8.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [120]:
%%timeit
(lf.filter(pl.col("cat_col_1") == 'A',
          pl.col('cat_col_2') == 'B')
 .select(pl.col(num_cols).mean()).collect())

33.1 ms ± 6.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Grupperinger og aggregering

In [121]:
%%timeit
df_np.groupby(cat_cols, observed=False)[num_cols].sum() 

538 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
%%timeit
df_arrow.groupby(cat_cols, observed=False)[num_cols].sum() 

530 ms ± 31.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [125]:
%%timeit
pf.group_by(cat_cols).agg(pl.col(num_cols).sum())

293 ms ± 42.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [127]:
%%timeit
lf.group_by(cat_cols).agg(pl.col(num_cols).sum()).collect()

272 ms ± 8.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [122]:
%%timeit
df_np.groupby(cat_cols, observed=False)[str_cols].agg('count')

1.94 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [124]:
%%timeit
df_arrow.groupby(cat_cols, observed=False)[str_cols].agg('count')

385 ms ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [126]:
%%timeit
pf.group_by(cat_cols).agg(pl.col(str_cols).count()) 

136 ms ± 8.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [128]:
%%timeit
lf.group_by(cat_cols).agg(pl.col(str_cols).count()).collect() 

136 ms ± 6.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
